# Databricks notebook: Gold - Focos Queimadas Agregado Diário

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, lit
spark = SparkSession.builder.getOrCreate()

In [0]:
# ================================================================
# Parâmetros
# ================================================================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_silver", "")
dbutils.widgets.text("table_silver", "")
dbutils.widgets.text("schema_gold", "")
dbutils.widgets.text("table_gold", "")
dbutils.widgets.text("data_ref_carga", "")

catalog        = dbutils.widgets.get("catalog")
schema_silver  = dbutils.widgets.get("schema_silver")
table_silver   = dbutils.widgets.get("table_silver")
schema_gold    = dbutils.widgets.get("schema_gold")
table_gold     = dbutils.widgets.get("table_gold")
data_ref_carga = dbutils.widgets.get("data_ref_carga")

print(f"📅 Data de referência: {data_ref_carga}")
print(f"📦 Lendo {catalog}.{schema_silver}.{table_silver}")

In [0]:
# ================================================================
# Leitura da Silver
# ================================================================
df_silver = spark.table(f"{catalog}.{schema_silver}.{table_silver}") \
    .filter(col("data_ref_carga") == data_ref_carga)

In [0]:
# ================================================================
# Agregação
# ================================================================
df_gold = (
    df_silver
    .groupBy("estado", "bioma", "data_ref_carga")
    .agg(count("*").alias("qtd_focos"))
)

df_gold = df_gold.withColumn("data_ref_processamento", lit(data_ref_carga))

print(f"✅ Linhas agregadas: {df_gold.count()}")


In [0]:
# ================================================================
# Escrita Gold
# ================================================================
(
    df_gold.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"data_ref_carga = '{data_ref_carga}'")
    .partitionBy("data_ref_carga")
    .saveAsTable(f"{catalog}.{schema_gold}.{table_gold}")
)

print(f"💾 Dados agregados gravados em: {catalog}.{schema_gold}.{table_gold}")